In [1]:
import numpy as np
from random import choice

class simulate_data():

    def __init__(self, n,d,sim,n_train):
        super().__init__()
        self.n = n
        self.d = d
        self.sim = sim
        self.n_train = n_train
        
    def simulate_feature_vec(self, a=2):
        if self.sim == 6:
            self.x = x = np.random.uniform(0,1,(self.n,self.d))
        else:
            # Random states
            def get_states(random_state, low, high, size):
                rs = np.random.RandomState(random_state)
                states = rs.randint(low=low, high=high, size=size)
                return states 
            self.states1 = get_states(random_state=42, low=0, high=100000, size=self.d)
            self.states2 = get_states(random_state=1028, low=0, high=100000, size=self.d)
            # generate one sequence for raw covariance matrice
            def generate_seq(i):
                np.random.seed(self.states1[i])
                a = np.random.randn(self.d)
                np.random.seed(self.states2[i])
                b = np.random.randn(1)
                return a+b 
            # Generate random covariance matrice
            A = np.matrix([generate_seq(i) for i in range(self.d)])
            A = A*np.transpose(A)
            D_half = np.diag(np.diag(A)**(-0.5))
            C = D_half*A*D_half
            # Generate d-dimensional feature vector
            mean = np.zeros(self.d)
            cov = C
            self.x = np.random.multivariate_normal(mean, cov, self.n) # shape (n,d)

    def generate_y_act(self):
        
        def simulate_y1():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            beta = np.random.uniform(-5,5,self.d)
            mu0 = np.dot(self.x, beta) +5*(self.x[:,0] > 0.5) 
            mu1 = mu0 + 8*(self.x[:,1] > 0.1)
            # Calculate y
            self.y1 = mu1 + e1 # y1:treatement group #y0:control group  
            self.y0 = mu0 + e0

        def simulate_y2():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            beta0 = np.random.uniform(1,30,self.d)
            beta1 = np.random.uniform(1,30,self.d)
            mu0 = np.dot(self.x, beta0)
            mu1 = np.dot(self.x, beta1)
            # Calculate y
            self.y1 = mu1 + e1
            self.y0 = mu0 + e0

        def simulate_y3():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            effect = 4 / (1+np.exp(-12 * ((self.x[:,0]-1)/2))) * (1+np.exp(-12 * ((self.x[:,1]-1)/2)))
            mu1 = 0.5 * effect
            mu0 = -mu1
            # Calculate y
            self.y1 = mu1 + e1
            self.y0 = mu0 + e0

        def simulate_y4():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            beta = np.random.uniform(1,30,self.d)
            mu0 = np.dot(self.x, beta)
            mu1 = mu0
            # Calculate y
            self.y1 = mu1 + e1
            self.y0 = mu0 + e0

        def simulate_y5():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            beta = np.random.uniform(-15,15,self.d)
            beta_m_dim = min(self.d,5)
            beta_m = beta[0:beta_m_dim]
            mu0 = np.zeros(self.n)
            mu0[self.x[:,19] < -0.4] = np.dot(self.x[self.x[:,19] < 0.-0.4][:,0:beta_m_dim],beta_m)
            mu0[(self.x[:,19] < 0.4) & (self.x[:,19] >= -0.4)] = np.dot(self.x[(self.x[:,19] < 0.4) & (self.x[:,19] >= -0.4)][:,beta_m_dim:(2*beta_m_dim)],beta_m)
            mu0[self.x[:,19] >= 0.4] = np.dot(self.x[self.x[:,19] >= 0.4][:,2*beta_m_dim:3*beta_m_dim],beta_m)
            mu0 = np.dot(self.x, beta)
            mu1 = mu0
            # Calculate y
            self.y1 = mu1 + e1
            self.y0 = mu0 + e0
        
        def simulate_y6():
            # Generate error term matrice
            e1 = np.random.randn(self.n)
            e0 = np.random.randn(self.n)
            # Generate mu
            mu0 = 2 * self.x[:,0] - 1
            mu1 = mu0
            # Calculate y
            self.y1 = mu1 + e1
            self.y0 = mu0 + e0
    
        if self.sim == 0:
            simulate_y1()
        elif self.sim == 1:
            simulate_y1()
        elif self.sim == 2:
            simulate_y2()
        elif self.sim == 3:
            simulate_y3()
        elif self.sim == 4:
            simulate_y4()
        elif self.sim == 5:
            simulate_y5()
        elif self.sim == 6:
            simulate_y6()
    
    def simulate_assignment(self):
        if self.sim == 6:
            # Different propensity scores
            prop_list = 1/4 * (1+ np.random.beta(2,4,self.n))
            self.w = np.random.binomial(1,prop_list,self.n)
        if self.sim == 0:
            self.w = np.random.binomial(1,0.01,self.n)
        else:
            self.w = np.random.binomial(1,0.5,self.n)
        
    def generate_assigned_y(self):
        self.y_obs = self.y1*self.w - self.y0*(self.w-1)
        
    def calculate_effect(self):
        self.eff_act = self.y1 - self.y0
    
    def split_data(self):
        self.x_train= self.x[:self.n_train,:]
        self.y_obs_train =  self.y_obs[:self.n_train]
        self.eff_act_train = self.eff_act[:self.n_train]
        self.w_train = self.w[:self.n_train]

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define forward process
class M(nn.Module):
    def __init__(self, n_feature,n_hidden, n_output = 1):
        super(M, self).__init__() 
        self.hidden = nn.Linear(n_feature, n_hidden)
        self.predict = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        # Here the forward pass is simply a linear function
        x = F.relu(self.hidden(x))
        x = self.predict(x)
        
        return x

# Define Planar Flow
class PlanarFlow(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(1, dim))
        self.bias = nn.Parameter(torch.Tensor(1))
        self.scale = nn.Parameter(torch.Tensor(1, dim))
        self.tanh = nn.Tanh()
        self.reset_parameters()

    def reset_parameters(self):
        self.weight.data.uniform_(-0.01, 0.01)
        self.scale.data.uniform_(-0.01, 0.01)
        self.bias.data.uniform_(-0.01, 0.01)

    def forward(self, z):
        activation = F.linear(z, self.weight, self.bias)
        return z + self.scale * self.tanh(activation)

# A helper class to compute the determinant of the gradient of the planar flow transformation.
class PlanarFlowLogDetJacobian(nn.Module):
    def __init__(self, affine):
        super().__init__()
        self.weight = affine.weight
        self.bias = affine.bias
        self.scale = affine.scale
        self.tanh = affine.tanh

    def forward(self, z):
        activation = F.linear(z, self.weight, self.bias)
        psi = (1 - self.tanh(activation) ** 2) * self.weight
        det_grad = 1 + torch.mm(psi, self.scale.t())
        def safe_log(z):
            return torch.log(z + 1e-7)
        return safe_log(det_grad.abs())
    
# Define Normalizing Flow class
class NormalizingFlow(nn.Module):
    def __init__(self, dim, flow_length):
        super().__init__()
        self.fc1 = nn.Linear(dim, 30)
        self.fc2 = nn.Linear(30, dim)
        
        self.transforms = nn.Sequential(*(
            PlanarFlow(dim) for _ in range(flow_length)))
        self.log_jacobians = nn.Sequential(*(
            PlanarFlowLogDetJacobian(t) for t in self.transforms))

    def forward(self, z):
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        
        log_jacobians = []
        for transform, log_jacobian in zip(self.transforms, self.log_jacobians):
            log_jacobians.append(log_jacobian(z))
            z = transform(z)
        zk = z

        return zk, log_jacobians

In [194]:
data =simulate_data(500000, 20, sim=6, n_train=200000) #0,1,6 #5000, 10000, 25000, 50000, 100000, 200000
data.simulate_feature_vec()
data.generate_y_act()
data.simulate_assignment()
data.generate_assigned_y()
data.calculate_effect()
data.split_data()

x_train = data.x_train
y_obs_train = data.y_obs_train
eff_act_train = data.eff_act_train  
w_train = data.w_train

x_trt_train = x_train[np.where(w_train == 1)]
y_obs_trt_train = y_obs_train[np.where(w_train == 1)]
w_trt_train = w_train[np.where(w_train == 1)]

x_con_train= x_train[np.where(w_train == 0)]
y_obs_con_train = y_obs_train[np.where(w_train == 0)]
w_con_train = w_train[np.where(w_train == 1)]

In [195]:
# Train M1 (treatment group)
x_m1 = torch.from_numpy(x_trt_train).float() # default is requires_grad=True, no need to add this any more if pytorch version >= 0.4
y_m1 = torch.from_numpy(y_obs_trt_train).float().unsqueeze(1)

# Define model
M_model1 = M(n_feature = 20,n_hidden =40, n_output =1)
criterion = torch.nn.MSELoss()
optimizer1 = torch.optim.SGD(M_model1.parameters(), lr = 0.01) #Stochastic Gradient Descent

# Training
for epoch in range(500):
    outputs = M_model1(x_m1) 
    loss = criterion(outputs, y_m1)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 1.382171869277954
epoch 100, loss 1.287340521812439
epoch 200, loss 1.242877721786499
epoch 300, loss 1.1855864524841309
epoch 400, loss 1.1232715845108032


In [196]:
# Train M2 (control group)
x_m0 = torch.from_numpy(x_con_train).float()
y_m0 = torch.from_numpy(y_obs_con_train).float().unsqueeze(1)

# Define model, loss function and optimizer
M_model0 = M(n_feature = 20,n_hidden = 40, n_output = 1)
criterion = torch.nn.MSELoss()
optimizer0 = torch.optim.SGD(M_model0.parameters(), lr = 0.01) #Stochastic Gradient Descent

# Training
for epoch in range(500):
    outputs = M_model0(x_m0) 
    loss = criterion(outputs, y_m0)

    optimizer0.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer0.step() # update gradients

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 1.3780337572097778
epoch 100, loss 1.3005287647247314
epoch 200, loss 1.2565492391586304
epoch 300, loss 1.2031644582748413
epoch 400, loss 1.1436454057693481


In [197]:
# Predict y0 for treatment group, y1 for control group
y1_con_learnt = M_model1(x_m0).data.numpy()
y0_trt_leant = M_model0(x_m1).data.numpy()

# Impute D1, D0
D1 = y_m1.data.numpy() - y0_trt_leant
D0 = y1_con_learnt - y_m0.data.numpy()

In [198]:
# Train f1(t,x) (treatment group)
t_x_trt_train = torch.from_numpy(x_trt_train).float()
t_D1 = torch.from_numpy(D1).float()
t_eff = torch.from_numpy(eff_act_train[np.where(w_train == 1)]).float().unsqueeze(1)

x_f1 = torch.cat((t_x_trt_train, t_D1),1)
y_f1 = torch.cat((t_x_trt_train, t_eff),1)

# Define Model
M_flow1 = NormalizingFlow(dim=21, flow_length=16)
optimizer = torch.optim.RMSprop(M_flow1.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_flow1(x_f1) 
    loss = criterion(outputs, y_f1)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 0.33990901708602905
epoch 3, loss 0.33990901708602905
epoch 6, loss 0.33990901708602905
epoch 9, loss 0.33990901708602905


In [199]:
# Train f0(t,x) (control group)
t_x_con_train = torch.from_numpy(x_con_train).float()
t_D0 = torch.from_numpy(D0).float()
t_eff_0 = torch.from_numpy(eff_act_train[np.where(w_train == 0)]).float().unsqueeze(1)

x_f0 = torch.cat((t_x_con_train, t_D0),1)
y_f0 = torch.cat((t_x_con_train, t_eff_0),1)

# Define Model
M_flow0 = NormalizingFlow(dim=21, flow_length=16)
optimizer = torch.optim.RMSprop(M_flow0.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_flow0(x_f0) 
    loss = criterion(outputs, y_f0)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 0.3772609233856201
epoch 3, loss 0.3772609233856201
epoch 6, loss 0.3772609233856201
epoch 9, loss 0.3772609233856201


In [200]:
# Train f1(x)
x_1 = torch.from_numpy(x_trt_train).float()

# Define Model
M_evd1 = NormalizingFlow(dim=20, flow_length=16)
optimizer = torch.optim.RMSprop(M_evd1.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_evd1(x_1) 
    loss = criterion(outputs, x_1)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 0.286047101020813
epoch 3, loss 0.286047101020813
epoch 6, loss 0.286047101020813
epoch 9, loss 0.286047101020813


In [201]:
# Train f0(x) (control group)\
x_0 = torch.from_numpy(x_con_train).float()

# Define Model
M_evd0 = NormalizingFlow(dim=20, flow_length=16)
optimizer = torch.optim.RMSprop(M_evd0.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_evd0(x_0) 
    loss = criterion(outputs, x_0)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 0.2748730480670929
epoch 3, loss 0.2748730480670929
epoch 6, loss 0.2748730480670929
epoch 9, loss 0.2748730480670929


In [203]:
t_x_trt_train = torch.from_numpy(x_trt_train).float()
t_D1 = torch.from_numpy(D1).float()
x_1 = torch.cat((t_x_trt_train, t_D1),1)
t_eff = torch.from_numpy(eff_act_train[np.where(w_train == 1)]).float().unsqueeze(1)
y_1 = torch.cat((t_x_trt_train, t_eff),1)

t_x_con_train = torch.from_numpy(x_con_train).float()
t_D0 = torch.from_numpy(D0).float()
x_0 = torch.cat((t_x_con_train, t_D0),1)
t_eff_0 = torch.from_numpy(eff_act_train[np.where(w_train == 0)]).float().unsqueeze(1)
y_0 = torch.cat((t_x_con_train, t_eff_0),1)

x = torch.cat((x_1, x_0),0)
x_b = torch.from_numpy(x_train).float()

f1_t_x,log1 = M_flow1(x)#.data.numpy() #
f0_t_x,log0 = M_flow0(x)#.data.numpy() #[:,-1]
f1_x = M_evd1(x_b)
f0_x = M_evd0(x_b)

t1 = f1_t_x.data.numpy()#[:,-1]
t0 = f0_t_x.data.numpy()#[:,-1]

g = 0.5
outputs = (g*t1 + (1-g)*t0) 

effect = np.concatenate((eff_act_train[np.where(w_train == 1)], eff_act_train[np.where(w_train == 0)]),0)

eff = torch.cat((y_1, y_0),0)
eff = eff.data.numpy()

MSE = (np.square(outputs - eff)).mean(axis=None)
MSE

0.3510784

In [204]:
## x-nn

In [205]:
D1.shape

(99804, 1)

In [206]:
# Train M1 (treatment group)
x_m1 = torch.from_numpy(x_trt_train).float() # default is requires_grad=True, no need to add this any more if pytorch version >= 0.4
y_m1 = torch.from_numpy(D1).float()

# Define model
M_3 = M(n_feature = 20,n_hidden =40, n_output =1)
criterion = torch.nn.MSELoss()
optimizer1 = torch.optim.SGD(M_model1.parameters(), lr = 0.01) #Stochastic Gradient Descent

# Training
for epoch in range(500):
    outputs = M_3(x_m1) 
    loss = criterion(outputs, y_m1)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 1.1319682598114014
epoch 100, loss 1.1319682598114014
epoch 200, loss 1.1319682598114014
epoch 300, loss 1.1319682598114014
epoch 400, loss 1.1319682598114014


In [207]:
# Train M2 (control group)
x_m0 = torch.from_numpy(x_con_train).float()
y_m0 = torch.from_numpy(D0).float()

# Define model, loss function and optimizer
M_4 = M(n_feature = 20,n_hidden = 40, n_output = 1)
criterion = torch.nn.MSELoss()
optimizer0 = torch.optim.SGD(M_model0.parameters(), lr = 0.01) #Stochastic Gradient Descent

# Training
for epoch in range(500):
    outputs = M_4(x_m0) 
    loss = criterion(outputs, y_m0)

    optimizer0.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer0.step() # update gradients

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 1.1540898084640503
epoch 100, loss 1.1540898084640503
epoch 200, loss 1.1540898084640503
epoch 300, loss 1.1540898084640503
epoch 400, loss 1.1540898084640503


In [208]:
x = torch.cat((x_1, x_0),0)
x_b = torch.from_numpy(x_train).float()

t1 = M_3(x_b).data.numpy()[:,-1]
t0 = M_4(x_b).data.numpy()[:,-1]

g = 0.5
outputs = (g*t1 + (1-g)*t0) 

eff = torch.cat((y_1, y_0),0)
eff = eff_act_train

MSE = (np.square(outputs - eff)).mean(axis=None)
MSE

2.0662393895778615

In [165]:
x-nn
5000 32.20489748888756 1.9970867649403918 
10000 32.82145760036449 2.0472418014862903
25000 30.141878200356224 0.36056638
50000 30.141878200356224 0.34985852
100000 31.830283975671097 2.000043453841319
200000 31.335946136683184 2.0662393895778615

SyntaxError: invalid syntax (<ipython-input-165-7ae8447f81e8>, line 2)

In [288]:
data =simulate_data(500000, 20, sim=0, n_train=5000) #0,1,6 #5000, 10000, 25000, 50000, 100000, 200000
data.simulate_feature_vec()
data.generate_y_act()
data.simulate_assignment()
data.generate_assigned_y()
data.calculate_effect()
data.split_data()

x_train = data.x_train
y_obs_train = data.y_obs_train
eff_act_train = data.eff_act_train  
w_train = data.w_train

x_trt_train = x_train[np.where(w_train == 1)]
y_obs_trt_train = y_obs_train[np.where(w_train == 1)]
w_trt_train = w_train[np.where(w_train == 1)]

x_con_train= x_train[np.where(w_train == 0)]
y_obs_con_train = y_obs_train[np.where(w_train == 0)]
w_con_train = w_train[np.where(w_train == 1)]

In [289]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_rf_0 = RandomForestRegressor(random_state=123, n_estimators=100)
X_rf_1 = RandomForestRegressor(random_state=123, n_estimators=100)
    
X_rf_0.fit(x_con_train, y_obs_con_train)
X_rf_1.fit(x_trt_train, y_obs_trt_train)
    
D1 = y_obs_trt_train - X_rf_0.predict(x_trt_train)
D0 = X_rf_1.predict(x_con_train) - y_obs_con_train

In [290]:
# Train f1(t,x) (treatment group)
t_x_trt_train = torch.from_numpy(x_trt_train).float()
t_D1 = torch.from_numpy(D1).float().unsqueeze(1)
t_eff = torch.from_numpy(eff_act_train[np.where(w_train == 1)]).float().unsqueeze(1)

x_f1 = torch.cat((t_x_trt_train, t_D1),1)
y_f1 = torch.cat((t_x_trt_train, t_eff),1)

# Define Model
M_flow1 = NormalizingFlow(dim=21, flow_length=16)
optimizer = torch.optim.RMSprop(M_flow1.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_flow1(x_f1) 
    loss = criterion(outputs, y_f1)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

epoch 0, loss 2.1755123138427734
epoch 3, loss 2.1755123138427734
epoch 6, loss 2.1755123138427734
epoch 9, loss 2.1755123138427734


C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [291]:
# Train f0(t,x) (control group)
t_x_con_train = torch.from_numpy(x_con_train).float()
t_D0 = torch.from_numpy(D0).float().unsqueeze(1)
t_eff_0 = torch.from_numpy(eff_act_train[np.where(w_train == 0)]).float().unsqueeze(1)

x_f0 = torch.cat((t_x_con_train, t_D0),1)
y_f0 = torch.cat((t_x_con_train, t_eff_0),1)

# Define Model
M_flow0 = NormalizingFlow(dim=21, flow_length=16)
optimizer = torch.optim.RMSprop(M_flow0.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training
for epoch in range(10):
    outputs,log_jacobians = M_flow0(x_f0) 
    loss = criterion(outputs, y_f0)

    optimizer1.zero_grad() # clear gradients for next train
    loss.backward() # backprops
    optimizer1.step() # update gradients

    if epoch % 3 == 0:
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 2.7235121726989746
epoch 3, loss 2.7235121726989746
epoch 6, loss 2.7235121726989746
epoch 9, loss 2.7235121726989746


In [292]:
t_x_trt_train = torch.from_numpy(x_trt_train).float()
t_D1 = torch.from_numpy(D1).float().unsqueeze(1)
x_1 = torch.cat((t_x_trt_train, t_D1),1)
t_eff = torch.from_numpy(eff_act_train[np.where(w_train == 1)]).float().unsqueeze(1)
y_1 = torch.cat((t_x_trt_train, t_eff),1)

t_x_con_train = torch.from_numpy(x_con_train).float()
t_D0 = torch.from_numpy(D0).float().unsqueeze(1)
x_0 = torch.cat((t_x_con_train, t_D0),1)
t_eff_0 = torch.from_numpy(eff_act_train[np.where(w_train == 0)]).float().unsqueeze(1)
y_0 = torch.cat((t_x_con_train, t_eff_0),1)

x = torch.cat((x_1, x_0),0)
x_b = torch.from_numpy(x_train).float()

f1_t_x,log1 = M_flow1(x)#.data.numpy() #
f0_t_x,log0 = M_flow0(x)#.data.numpy() #[:,-1]
f1_x = M_evd1(x_b)
f0_x = M_evd0(x_b)

t1 = f1_t_x.data.numpy()#[:,-1]
t0 = f0_t_x.data.numpy()#[:,-1]

g = 0.5
outputs = (g*t1 + (1-g)*t0) 

effect = np.concatenate((eff_act_train[np.where(w_train == 1)], eff_act_train[np.where(w_train == 0)]),0)

eff = torch.cat((y_1, y_0),0)
eff = eff.data.numpy()

MSE = (np.square(outputs - eff)).mean(axis=None)
MSE

2.4854677

In [293]:
5000 2.4854677 0.36664274
10000 2.5352986 0.32686228
25000 2.4162445 0.35771015
50000 2.4621582 0.35529676
100000 2.4257748 0.32686228
200000 2.4355667 0.35802394

SyntaxError: invalid syntax (<ipython-input-293-3cbab3aff68d>, line 1)